## Setup

In [ ]:
# GPU Configuration - Use P100 (GPU 1) for best performance
import os

# Set library path for CUDA libraries installed via pip
venv_cuda_libs = '/mnt/arkk/kaggle/diabetes-prediction/.venv/lib/python3.12/site-packages/nvidia/cudnn/lib'

if 'LD_LIBRARY_PATH' in os.environ:
    os.environ['LD_LIBRARY_PATH'] = f"{os.environ['LD_LIBRARY_PATH']}:{venv_cuda_libs}"

else:
    os.environ['LD_LIBRARY_PATH'] = venv_cuda_libs

os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # 1=GTX1080, 0=P100
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Verify GPU
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print(f"TensorFlow version: {tf.__version__}")
print(f"GPUs available: {gpus}")
print(f"Number of GPUs: {len(gpus)}")

2025-12-08 18:42:47.498885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765237367.525941 2982339 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765237367.533347 2982339 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Standard library imports
import os
import sys
import time
import warnings
from pathlib import Path
from glob import glob

# Third-party imports
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow/Keras imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers, callbacks

# Keras Tuner
from keras_tuner import RandomSearch, Objective

# Scikit-learn
from sklearn.metrics import roc_auc_score

# Add functions to path
sys.path.insert(0, str(Path('.').resolve()))

# Import ensemble modules
from functions.ensemble_initialization import create_data_splits

### Configuration

In [ ]:
# Configuration
RANDOM_STATE = 315
LABEL = 'diagnosed_diabetes'

# Hyperparameter search setting
MAX_TRIALS = 8            # Slightly more trials for better search
EXECUTIONS_PER_TRIAL = 1  # More executions for statistical reliability
MAX_EPOCHS = 50           # Full epochs for thorough testing
SAMPLE_SIZE = 0.30        # 50% sample


## Data Preparation

In [ ]:
# Load training data
train_df_path = 'https://gperdrizet.github.io/FSA_devops/assets/data/unit3/diabetes_prediction_train.csv'
train_df = pd.read_csv(train_df_path)
train_df.drop_duplicates(inplace=True)

print(f'Training data shape: {train_df.shape}')
print(f'Class distribution:')
print(train_df[LABEL].value_counts(normalize=True))

In [ ]:
# Create 60/35/5 split (same as hill climbing)
X_train_pool, X_val_s1, X_val_s2, y_train_pool, y_val_s1, y_val_s2 = create_data_splits(
    train_df, LABEL, RANDOM_STATE
)

print(f"\nData splits (same as hill climbing):")
print(f"  X_train_pool: {X_train_pool.shape} (60%)")
print(f"  X_val_s1: {X_val_s1.shape} (35%)")
print(f"  X_val_s2: {X_val_s2.shape} (5%)")

In [ ]:
# Sample data for optimization
sample_size = int(len(X_val_s1) * SAMPLE_SIZE)
X_val_s1_sample = X_val_s1.iloc[:sample_size]
y_val_s1_sample = y_val_s1.iloc[:sample_size]

sample_size_s2 = int(len(X_val_s2) * SAMPLE_SIZE)
X_val_s2_sample = X_val_s2.iloc[:sample_size_s2]
y_val_s2_sample = y_val_s2.iloc[:sample_size_s2]

print(f"Using {SAMPLE_SIZE * 100:.0f}% sample for optimization:")
print(f"  X_val_s1_sample: {X_val_s1_sample.shape} ({SAMPLE_SIZE * 100:.0f}% of {X_val_s1.shape[0]:,})")
print(f"  X_val_s2_sample: {X_val_s2_sample.shape} ({SAMPLE_SIZE * 100:.0f}% of {X_val_s2.shape[0]:,})")
print()

## Load stage 1 models

In [ ]:
models_path = f"../models/run_20251208_045148/ensemble_stage1_models"

print(f"Loading models from: {models_path}\n")

# Find all model files (excluding founder)
model_files = sorted(glob(f"{models_path}/model_*.joblib"))

# Load first 5 models
stage1_models = []

for model_file in model_files[:5]:
    model_name = Path(model_file).stem
    model = joblib.load(model_file)
    
    # Evaluate on validation set
    # Handle both predict_proba and decision_function (e.g., LinearSVC)
    if hasattr(model, 'predict_proba'):
        pred = model.predict_proba(X_val_s1)[:, 1]
    else:
        # Use decision_function for models like LinearSVC
        pred = model.decision_function(X_val_s1)
    
    auc = roc_auc_score(y_val_s1, pred)
    
    stage1_models.append(model)
    print(f"  {model_name}: AUC = {auc:.6f}")

print(f"\n{len(stage1_models)} Stage 1 models loaded!")
print(f"Model files: {[Path(f).stem for f in model_files[:5]]}")

In [ ]:
# Generate Stage 1 predictions
all_stage1_preds_s1 = []
for model in stage1_models:
    if hasattr(model, 'predict_proba'):
        pred = model.predict_proba(X_val_s1_sample)[:, 1]
    else:
        pred = model.decision_function(X_val_s1_sample)
    all_stage1_preds_s1.append(pred)

all_stage1_preds_s2 = []
for model in stage1_models:
    if hasattr(model, 'predict_proba'):
        pred = model.predict_proba(X_val_s2_sample)[:, 1]
    else:
        pred = model.decision_function(X_val_s2_sample)
    all_stage1_preds_s2.append(pred)

X_stage2_s1 = np.column_stack(all_stage1_preds_s1)
X_stage2_s2 = np.column_stack(all_stage1_preds_s2)
y_stage2_s1 = y_val_s1_sample.values
y_stage2_s2 = y_val_s2_sample.values

# 90/10 split
split_idx = int(len(X_stage2_s2) * 0.9)
X_train_opt = np.vstack([X_stage2_s1, X_stage2_s2[:split_idx]])
y_train_opt = np.concatenate([y_stage2_s1, y_stage2_s2[:split_idx]])
X_val_opt = X_stage2_s2[split_idx:]
y_val_opt = y_stage2_s2[split_idx:]

print(f"Training samples: {len(X_train_opt):,}")
print(f"Validation samples: {len(X_val_opt):,}")
print()

## Model definition

In [ ]:
# OFFLINE OPTIMIZATION parameters
MAX_TRIALS = 15           # Slightly more trials for better search
EXECUTIONS_PER_TRIAL = 2  # More executions for statistical reliability
MAX_EPOCHS = 100          # Full epochs for thorough testing
SAMPLE_SIZE = 0.50        # 50% sample

def build_regularized_model(hp, n_models):
    """Build model with strong regularization to prevent overfitting."""

    # Simpler architectures - prefer fewer, smaller layers
    arch_type = hp.Choice('architecture_type', values=['funnel', 'constant', 'pyramid'])
    n_layers = hp.Int('n_layers', min_value=1, max_value=3)
    base_units = hp.Choice('base_units', values=[16, 32, 64, 128])
    
    # Strong regularization
    dropout = hp.Float('dropout', min_value=0.2, max_value=0.7)
    l2_reg = hp.Float('l2_reg', min_value=1e-4, max_value=1e-2, sampling='log')
    learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-4, sampling='log')
    
    # Generate layer units based on architecture type
    if arch_type == 'funnel':
    
        # Decreasing units: [base, base//2, base//4, ...]
        if n_layers == 1:
            units_per_layer = [base_units]

        elif n_layers == 2:
            units_per_layer = [base_units, base_units // 2]

        else:  # n_layers == 3
            units_per_layer = [base_units, base_units // 2, base_units // 4]
    
    elif arch_type == 'constant':

        # Same units per layer: [base, base, base, ...]
        units_per_layer = [base_units] * n_layers
    
    elif arch_type == 'pyramid':

        # Increasing then decreasing: [base//2, base, base//2] or similar
        if n_layers == 1:
            units_per_layer = [base_units]

        elif n_layers == 2:
            units_per_layer = [base_units // 2, base_units]

        else:  # n_layers == 3
            units_per_layer = [base_units // 2, base_units, base_units // 2]
    
    # Build model
    model = models.Sequential()
    model.add(layers.Input(shape=(n_models,)))
    
    for units in units_per_layer:
        model.add(layers.Dense(
            units, 
            activation='relu',
            kernel_regularizer=regularizers.l2(l2_reg)
        ))
        model.add(layers.Dropout(dropout))
    
    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC(name='auc')]
    )
    
    return model, {
        'architecture_type': arch_type,
        'n_layers': n_layers,
        'base_units': base_units,
        'units_per_layer': units_per_layer,
        'dropout': dropout,
        'l2_reg': l2_reg,
        'learning_rate': learning_rate
    }


def build_model_from_config(config, n_models):
    """Build model from hyperparameter configuration (for retraining)."""
    
    # Extract hyperparameters
    arch_type = config['architecture_type']
    n_layers = config['n_layers']
    base_units = config['base_units']
    dropout = config['dropout']
    l2_reg = config['l2_reg']
    learning_rate = config['learning_rate']
    
    # Generate layer units based on architecture type
    if arch_type == 'funnel':
        if n_layers == 1:
            units_per_layer = [base_units]
        elif n_layers == 2:
            units_per_layer = [base_units, base_units // 2]
        else:  # n_layers == 3
            units_per_layer = [base_units, base_units // 2, base_units // 4]
    
    elif arch_type == 'constant':
        units_per_layer = [base_units] * n_layers
    
    elif arch_type == 'pyramid':
        if n_layers == 1:
            units_per_layer = [base_units]
        elif n_layers == 2:
            units_per_layer = [base_units // 2, base_units]
        else:  # n_layers == 3
            units_per_layer = [base_units // 2, base_units, base_units // 2]
    
    # Build model
    model = models.Sequential()
    model.add(layers.Input(shape=(n_models,)))
    
    for units in units_per_layer:
        model.add(layers.Dense(
            units,
            activation='relu',
            kernel_regularizer=regularizers.l2(l2_reg)
        ))
        model.add(layers.Dropout(dropout))
    
    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC(name='auc'), 'accuracy']
    )
    
    return model, units_per_layer


## Custom Regularized Optimization

Run custom hyperparameter search with anti-overfitting focus.

In [ ]:
# Suppress the optimizer loading warning
warnings.filterwarnings('ignore', message='Skipping variable loading for optimizer')

# Run custom optimization
n_models = X_train_opt.shape[1]
tuner_dir = Path('../models/keras_tuner')
tuner_dir.mkdir(parents=True, exist_ok=True)

def build_model_wrapper(hp):
    """Wrapper for Keras Tuner."""

    model, hps = build_regularized_model(hp, n_models)
    return model

tuner = RandomSearch(
    build_model_wrapper,
    objective=Objective('val_loss', direction='min'),  # Optimize val_loss, not AUC!
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTIONS_PER_TRIAL,
    directory=str(tuner_dir),
    project_name='stage2_regularized_tuning',
    overwrite=True
)

early_stop = callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor val_loss
    patience=10,
    mode='min',
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

start_time = time.time()

tuner.search(
    X_train_opt, y_train_opt,
    epochs=MAX_EPOCHS,  # Use configurable epochs
    validation_data=(X_val_opt, y_val_opt),
    callbacks=[early_stop, reduce_lr],
    verbose=1,
    batch_size=64  # Smaller batch size for better generalization
)

elapsed_time = time.time() - start_time

# Get best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Evaluate
y_pred_val = best_model.predict(X_val_opt, verbose=0).flatten()
val_auc = roc_auc_score(y_val_opt, y_pred_val)

# Store best hyperparameters for retraining
optimized_hps = {
    'architecture_type': best_hps.get('architecture_type'),
    'n_layers': best_hps.get('n_layers'),
    'base_units': best_hps.get('base_units'),
    'dropout': best_hps.get('dropout'),
    'l2_reg': best_hps.get('l2_reg'),
    'learning_rate': best_hps.get('learning_rate')
}

print(f"\nElapsed time: {elapsed_time/60:.1f} minutes ({elapsed_time/3600:.1f} hours)")
print(f"Best validation AUC: {val_auc:.6f}")
print(f"\nBest Hyperparameters:")
print(f"  Architecture: {best_hps.get('architecture_type')}")
print(f"  Layers: {best_hps.get('n_layers')}")
print(f"  Base units: {best_hps.get('base_units')}")
print(f"  Dropout: {best_hps.get('dropout'):.3f}")
print(f"  L2 reg: {best_hps.get('l2_reg'):.6f}")
print(f"  Learning rate: {best_hps.get('learning_rate'):.6f}")

## Retrain on Full Dataset

Now retrain the best model on the complete dataset to get final performance and learning curves.

In [ ]:
# Generate Stage 1 predictions on ALL data (both validation sets)
print("Generating Stage 1 predictions on full validation set...")

all_stage1_preds_s1 = []

for model in stage1_models:

    if hasattr(model, 'predict_proba'):
        pred = model.predict_proba(X_val_s1)[:, 1]

    else:
        pred = model.decision_function(X_val_s1)

    all_stage1_preds_s1.append(pred)

all_stage1_preds_s2 = []

for model in stage1_models:

    if hasattr(model, 'predict_proba'):
        pred = model.predict_proba(X_val_s2)[:, 1]

    else:
        pred = model.decision_function(X_val_s2)

    all_stage1_preds_s2.append(pred)

# Stack predictions
X_stage2_s1 = np.column_stack(all_stage1_preds_s1)
X_stage2_s2 = np.column_stack(all_stage1_preds_s2)
y_stage2_s1 = y_val_s1.values
y_stage2_s2 = y_val_s2.values

# Use 90/10 split for train/val
split_idx = int(len(X_stage2_s2) * 0.9)
X_stage2_s2_train = X_stage2_s2[:split_idx]
X_stage2_s2_val = X_stage2_s2[split_idx:]
y_stage2_s2_train = y_stage2_s2[:split_idx]
y_stage2_s2_val = y_stage2_s2[split_idx:]

X_train_full = np.vstack([X_stage2_s1, X_stage2_s2_train])
y_train_full = np.concatenate([y_stage2_s1, y_stage2_s2_train])
X_val_full = X_stage2_s2_val
y_val_full = y_stage2_s2_val

print(f"Full training set: {len(X_train_full):,} samples")
print(f"Full validation set: {len(X_val_full):,} samples")

In [ ]:
# Build fresh model with optimized regularized hyperparameters using reusable function
n_models = X_train_full.shape[1]

final_model, units_per_layer = build_model_from_config(optimized_hps, n_models)

print("Regularized Model Architecture:")
print(f"  Architecture: {optimized_hps['architecture_type']}")
print(f"  Layers: {units_per_layer}")
print(f"  Dropout: {optimized_hps['dropout']:.3f}")
print(f"  L2 reg: {optimized_hps['l2_reg']:.6f}")
print(f"  Learning rate: {optimized_hps['learning_rate']:.6f}")
print()
final_model.summary()


In [ ]:
# Train final model with early stopping on VALIDATION LOSS

# Monitor val_loss instead of val_auc to prevent overfitting
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,  # More patience for val_loss
    mode='min',
    restore_best_weights=True,
    verbose=1
)

# Add ReduceLROnPlateau for adaptive learning rate
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    mode='min',
    min_lr=1e-7,
    verbose=1
)

# Track learning rate for plotting
lr_tracker = callbacks.LearningRateScheduler(lambda epoch, lr: lr, verbose=0)

print("\nTraining final regularized model on full dataset...")
print("=" * 80)

history = final_model.fit(
    X_train_full, y_train_full,
    epochs=200,
    batch_size=64,
    validation_data=(X_val_full, y_val_full),
    callbacks=[early_stop, reduce_lr, lr_tracker],
    verbose=1
)

print("=" * 80)
print("Training complete!")


In [ ]:
# Evaluate final model
y_pred = final_model.predict(X_val_full, verbose=0).flatten()
final_auc = roc_auc_score(y_val_full, y_pred)

# Find best epoch (where val_loss was minimum)
best_epoch = np.argmin(history.history['val_loss']) + 1

print(f"\nFinal Regularized Model Performance:")
print(f"  Validation AUC: {final_auc:.6f}")
print(f"  Total epochs trained: {len(history.history['loss'])}")
print(f"  Best epoch (min val_loss): {best_epoch}")
print(f"  Final train loss: {history.history['loss'][-1]:.6f}")
print(f"  Final val loss: {history.history['val_loss'][-1]:.6f}")
print(f"  Best val loss: {min(history.history['val_loss']):.6f}")

## Learning Curves

In [ ]:
# Find best epoch
best_epoch_idx = np.argmin(history.history['val_loss'])

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Loss (MOST IMPORTANT - should show both decreasing)
axes[0, 0].plot(history.history['loss'], label='Training Loss', linewidth=2, color='blue')
axes[0, 0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2, color='orange')
axes[0, 0].axvline(x=best_epoch_idx, color='red', linestyle='--', alpha=0.7, label='Best Epoch')
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Loss', fontsize=12)
axes[0, 0].set_title('Loss Over Training (Key: Both Should Decrease)', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: AUC
axes[0, 1].plot(history.history['auc'], label='Training AUC', linewidth=2, color='blue')
axes[0, 1].plot(history.history['val_auc'], label='Validation AUC', linewidth=2, color='orange')
axes[0, 1].axvline(x=best_epoch_idx, color='red', linestyle='--', alpha=0.7, label='Best Epoch')
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('AUC', fontsize=12)
axes[0, 1].set_title('AUC Over Training', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Accuracy
axes[1, 0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2, color='blue')
axes[1, 0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2, color='orange')
axes[1, 0].axvline(x=best_epoch_idx, color='red', linestyle='--', alpha=0.7, label='Best Epoch')
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Accuracy', fontsize=12)
axes[1, 0].set_title('Accuracy Over Training', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Learning Rate Schedule
if 'lr' in history.history:
    axes[1, 1].plot(history.history['lr'], linewidth=2, color='green')
    axes[1, 1].axvline(x=best_epoch_idx, color='red', linestyle='--', alpha=0.7, label='Best Epoch')
    axes[1, 1].set_xlabel('Epoch', fontsize=12)
    axes[1, 1].set_ylabel('Learning Rate', fontsize=12)
    axes[1, 1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[1, 1].set_yscale('log')
    axes[1, 1].legend(fontsize=10)
    axes[1, 1].grid(True, alpha=0.3)
else:
    axes[1, 1].text(0.5, 0.5, 'Learning Rate Not Tracked', 
                    ha='center', va='center', fontsize=14)
    axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("TRAINING ANALYSIS")
print("=" * 80)
print(f"Best epoch (min val_loss): {best_epoch_idx + 1}")
print(f"Training loss at best epoch: {history.history['loss'][best_epoch_idx]:.6f}")
print(f"Validation loss at best epoch: {history.history['val_loss'][best_epoch_idx]:.6f}")
print(f"Gap (train - val): {history.history['loss'][best_epoch_idx] - history.history['val_loss'][best_epoch_idx]:.6f}")
print()
print(f"Best training AUC: {max(history.history['auc']):.6f}")
print(f"Best validation AUC: {max(history.history['val_auc']):.6f}")
print(f"Final validation AUC: {final_auc:.6f}")
print("=" * 80)

# Overfitting check
train_val_gap = history.history['loss'][best_epoch_idx] - history.history['val_loss'][best_epoch_idx]
if train_val_gap < 0.01:
    print("✅ GOOD: Minimal overfitting (train-val gap < 0.01)")
elif train_val_gap < 0.05:
    print("⚠️  MODERATE: Some overfitting (train-val gap 0.01-0.05)")
else:
    print("❌ BAD: Significant overfitting (train-val gap > 0.05)")
print("=" * 80)

## Summary

✅ **Test notebook complete!**

### What we did:

1. ✅ Loaded training data (60/20/20 split)
2. ✅ Loaded actual Stage 1 models from batch 1
3. ✅ Ran **`optimize_and_update_config()`** (same as online hill climbing)
4. ✅ Displayed optimized hyperparameters
5. ✅ Generated copy-paste config for `ensemble_config.py`

### Key points:

- **Shared code path**: Uses identical optimization function as online training
- **Real data**: Uses actual Stage 1 models from hill climbing run
- **GPU-optimized**: ~15-30 min on P100 GPU
- **Focused search**: pyramid/funnel architectures, 2-3 layers, 64-256 units
- **Conservative split**: 95% training (266k), 5% validation (14k)

### Next steps:

1. Copy the `STAGE2_DNN_CONFIG` above into `ensemble_config.py`
2. Run full hill climbing with GPU enabled
3. Architecture will be automatically optimized at batches 10, 20, 30+
4. Compare performance against baseline

## Summary: Optimization Speed Recommendations

### Observed Performance (This Notebook - Offline Testing)
- **Actual Runtime**: 220 minutes (3.7 hours) for 15 trials × 2 executions
- **Per Trial**: ~7.3 minutes per trial on 50% sample
- **Settings**: 15 trials × 2 executions = 30 model trainings, 100 max epochs, 50% sample

### Recommended Online Settings (Hill Climbing Production)
- **Trials**: 8 trials × 1 execution = 8 model trainings
- **Max Epochs**: 50 (early stopping usually ~15-20 epochs)
- **Sample Size**: 30% of data (~1,050 training samples)
- **Estimated Time**: ~30-40 minutes
  - Based on 7.3 min/trial × 0.6 (sample reduction) = ~4.4 min/trial
  - 8 trials × 4.4 min ≈ **35 minutes**

### Speed Improvements for Online Optimization
1. **Reduce trials**: 15 → 8 trials
2. **Single execution**: 2 → 1 executions per trial
3. **Smaller sample**: 50% → 30% (0.6x speedup per trial)

**Total speedup**: ~6-7x faster (220 min → 35 min)

### Why This Works
- **8 trials** is sufficient for the small hyperparameter space (funnel-only, 1-2 layers, 2 unit choices)
- **Single execution** acceptable because early stopping provides regularization
- **50 max epochs** sufficient since early stopping typically triggers around epoch 15-30
- **30% sample** still provides ~1,050 training samples - adequate for 5-dimensional input

### Important Note
Original 5-hour estimate was based on incorrect assumptions. Actual observed performance shows each trial takes ~7 minutes, not <1 minute.